# Champions League Round of 16 Probability Simulation

In [1]:
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.utils import get_column_letter
import ipywidgets as widgets

### Load the candidates
Read Excel File and calculate possible matchups.

In [2]:
# WB_file  = '2021_CL.xlsx' 
# WB_file  = '1920_CL.xlsx' 
# WB_file  = '1819_CL.xlsx' 
WB_file  = '1213_CL.xlsx' 

In [3]:
Ro16df = pd.read_excel(WB_file, sheet_name='RoundOf16')
Ro16df.fillna('', inplace=True)
Seeded = Ro16df.iloc[4:13,2]
Seeded.iloc[0] = 'C'
Unseeded = Ro16df.iloc[3,3:12]
Unseeded.iloc[0] = 'C'
Ro16df = Ro16df.iloc[4:13,3:12]
Ro16df.iloc[1:9,1:9] = 0
Ro16df = Ro16df.rename(columns=Unseeded, index=Seeded)
for k in range(1,9):
    for l in range(1,9):
        if k != l and Ro16df.iloc[k,0] != Ro16df.iloc[0,l]:
            Ro16df.iloc[k,l] = 1
Ro16df

,C,Porto,Arsenal,Milan,Real Madrid,Shakhtar,Valencia,Celtic,Galatasaray
C,,PO,EN,IT,ES,UK,ES,SC,TU
Paris SG,FR,0,1,1,1,1,1,1,1
Schalke,DE,1,0,1,1,1,1,1,1
Malaga,ES,1,1,0,0,1,0,1,1
Dortmund,DE,1,1,1,0,1,1,1,1
Juventus,IT,1,1,0,1,0,1,1,1
Bayern,DE,1,1,1,1,1,0,1,1
Barcelona,ES,1,1,1,0,1,0,0,1
Man United,EN,1,0,1,1,1,1,1,0


In [4]:
Ro16df.drop('C', inplace=True, axis=0)
Ro16df.drop('C', inplace=True, axis=1)
Ro16df.columns
Ro16df.shape
Ro16df
#Ro16df = Ro16df.dropna(axis=0, how='all')
#Ro16df = Ro16df.dropna(axis=1, how='all')

,Porto,Arsenal,Milan,Real Madrid,Shakhtar,Valencia,Celtic,Galatasaray
Paris SG,0,1,1,1,1,1,1,1
Schalke,1,0,1,1,1,1,1,1
Malaga,1,1,0,0,1,0,1,1
Dortmund,1,1,1,0,1,1,1,1
Juventus,1,1,0,1,0,1,1,1
Bayern,1,1,1,1,1,0,1,1
Barcelona,1,1,1,0,1,0,0,1
Man United,1,0,1,1,1,1,1,0


### Calculate probabilities

In [5]:
PossSum = np.zeros((8,8))
CurrentComb = np.zeros(8, dtype=int)
Count = 0
Level = 0
Backw = False

In [6]:
Ro16np = Ro16df.to_numpy()

# Create a list of the possible cominations matrix
Ro16npComb = [np.copy(Ro16np)]
for k in range(7):
    Ro16npComb.append(np.copy(Ro16np))

In [7]:

while Level > -1:

    if Level == 8:
        Backw = True
    else:
        Backw = False

    if not Backw:    
        idx = np.where(Ro16npComb[Level][Level,:])
        if idx[0].size == 0:
            Backw = True

    while Backw:
        Level -= 1
        if Level > -1:
            idx = np.where(Ro16npComb[Level][Level,:])
            if idx[0].size:
                if Level < 7:
                    CurrentComb[Level+1:] = -1
                Backw = False
                if Level < 8:
                    if Level > 0:
                        Ro16npComb[Level][Level+1:,CurrentComb[Level]] = np.copy(Ro16npComb[Level-1][Level+1:,CurrentComb[Level]])
                        if Level < 7:
                            for l in range(Level+1,8):
                                Ro16npComb[l] = np.copy(Ro16npComb[Level])
                    else:
                        Ro16npComb[Level][1:,CurrentComb[0]] = np.copy(Ro16np[1:,CurrentComb[0]])
                        for l in range(1,8):
                            Ro16npComb[l] = np.copy(Ro16npComb[Level])
        else:
            break

    if idx[0].size:
        CurrentComb[Level] = idx[0][0]
        for l in range(Level,8):
            Ro16npComb[l][Level:,CurrentComb[Level]] = 0
        Level += 1
        if Level == 8:
            Count += 1
            for l in range(8):
                PossSum[l,CurrentComb[l]] += 1
    
print('Number of Combinations = ', Count)     

Number of Combinations =  5463


In [8]:
# PossSum = PossSum/Count
for k in range(8):
    for l in range(8):
        Ro16df.iloc[k,l] = PossSum[k,l]/Count
Ro16df

,Porto,Arsenal,Milan,Real Madrid,Shakhtar,Valencia,Celtic,Galatasaray
Paris SG,0,0.130514,0.144792,0.183965,0.116786,0.183965,0.123558,0.11642
Schalke,0.11642,0,0.146806,0.187809,0.11825,0.187809,0.125206,0.117701
Malaga,0.189639,0.222222,0,0,0.193117,0,0.201355,0.193666
Dortmund,0.123742,0.14168,0.155409,0,0.126121,0.195497,0.131979,0.125572
Juventus,0.133443,0.151382,0,0.217646,0,0.217646,0.144792,0.135091
Bayern,0.123742,0.14168,0.155409,0.195497,0.126121,0,0.131979,0.125572
Barcelona,0.1825,0.212521,0.231924,0,0.187077,0,0,0.185978
Man United,0.130514,0,0.16566,0.215083,0.132528,0.215083,0.141131,0


### Export to Excel

In [9]:
wb = openpyxl.load_workbook(WB_file)
for l in range(8):
    for k in range(8):
        wb['RoundOf16'][get_column_letter(l+5) + str(k+7)] = Ro16df.iloc[k,l]

wb['RoundOf16']['G16'] = Count
wb.save(WB_file[0:-5]+'_Prob.xlsx')
wb.close()

C:\Users\marsc\anaconda3\envs\work02\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
